In [ ]:
import JupyterNotebooksLib as slicernb

# Prepare screen

In [ ]:
# Clear scene
slicer.mrmlScene.Clear(False)

# Set up view layout
slicer.app.layoutManager().setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView)

In [ ]:
import os
from collections import namedtuple
import slicer


class Callback(object):
    def __init__(self, on_update=None):
        self.on_update = on_update or (lambda *args, **kwargs: None)

### Configuração

Para fins de detalhamento, eu trouxe a implementação do objeto de parametrização para dentro do Notebook, mas é possível acessá-lo via import do modulo. 

In [ ]:
DATASETS_DIRECTORY = "datasetsDirectory"
EXPORT_DIRECTORY = "exportDirectory"
INITIAL_DEPTH = "initialDepth"
CORE_LENGTH = "coreLength"
CORE_DIAMETER = "coreDiameter"
WELL_DIAMETER = "wellDiameter"
UNWRAP_LAYER_DEPTH = "unwrapRadialDepth"
ORIENTATION_ALGORITHM = "orientationAlgorithm"
ORIENTATION_ALGORITHM_NONE = 0
ORIENTATION_ALGORITHM_SURFACE = 1
ORIENTATION_ALGORITHM_SINUSOID = 2
ORIENTATION_ALGORITHM_SURFACE_SINUSOID = 3
KEEP_ORIGINAL_VOLUMES = "keepOriginalVolumes"
CORE_RADIAL_CORRECTION = "coreRadialCorrection"
SMOOTH_CORE_SURFACE = "smoothCoreSurface"

LoadDatasetsParameters = namedtuple(
    "LoadDatasetsParameters", ["callback", DATASETS_DIRECTORY, INITIAL_DEPTH, CORE_LENGTH]
)

ProcessCoresParameters = namedtuple(
    "ProcessCoresParameters",
    ["callback", CORE_DIAMETER, CORE_RADIAL_CORRECTION, SMOOTH_CORE_SURFACE, KEEP_ORIGINAL_VOLUMES],
)

ApplyAllParameters = namedtuple(
    "ApplyAllParameters",
    [
        "callback",
        DATASETS_DIRECTORY,
        INITIAL_DEPTH,
        CORE_LENGTH,
        CORE_DIAMETER,
        CORE_RADIAL_CORRECTION,
        SMOOTH_CORE_SURFACE,
        KEEP_ORIGINAL_VOLUMES,
        ORIENTATION_ALGORITHM,
        UNWRAP_LAYER_DEPTH,
        WELL_DIAMETER,
    ],
)

### Chamando a função

Abaixo a gente cria um objeto que define a configuração para ler uma pasta contendo os DICOMs de uma tomografia de testemunho. Para rodar, é preciso trocar a pasta "./data/cores", pela pasta onde estão os DICOMs na sua máquina.

In [ ]:
from ltrace.units import global_unit_registry as ureg, SLICER_LENGTH_UNIT


def updateStatus(message, progress, processEvents):
    print(progress)


callback = Callback(
    on_update=lambda message, percent, processEvents=True: updateStatus(
        message,
        progress=percent,
        processEvents=processEvents,
    )
)

applyAllParameters = ApplyAllParameters(
    callback,
    "./data/cores",  # <----------- TROCAR AQUI colocar o caminho raiz dos diretorios dos testemunhos (em DICOM). Ex: ./data/cores/T02_CX01, ./data/cores/T02_CX02
    float(5422) * ureg.meter,
    float(90) * ureg.centimeter,
    float(130) * ureg.millimeter,
    True,
    True,
    True,
    "Surface + Sinusoid",
    float(4) * ureg.millimeter,
    float(308) * ureg.millimeter,
)

from Multicore import MulticoreLogic

logic = MulticoreLogic()
logic.applyAll(applyAllParameters)

In [ ]:
# Slice view display
from ipywidgets import interact


@interact(position=(0, 100))
def update(position=50):
    return slicernb.ViewSliceDisplay("Red", positionPercent=position)

In [ ]:
# Slice widgets
from ipywidgets import HBox

display(HBox([slicernb.ViewSliceWidget("Red"), slicernb.ViewSliceWidget("Yellow"), slicernb.ViewSliceWidget("Green")]))

In [ ]:
slicernb.View3DDisplay()

In [ ]:
# Set image viewer size to 50% (fill half of a cell)
slicernb.AppWindow.setWindowSize(scale=2)
slicernb.ViewInteractiveWidget()

In [ ]:
# Lightbox view
slicernb.ViewLightboxDisplay(
    "Red",
    rangeShrink=[45, 50],
)